# Model Interpretability

In this notebook we will try to understand our models. The goal is to go beyond machine learning metrics commonly used for evaluation and instead focus on other important aspects (see also `test.py`)

In [ ]:
from google.cloud import storage

import warnings
warnings.filterwarnings('ignore')

import pickle
from sklearn import metrics
import xgboost as xgb
import pandas as pd 
import numpy as np
import shap

from matplotlib import pyplot as plt
plt.style.use('ggplot')

storage_client = storage.Client()

# Change these if needed!
MODEL_PATH = 'train_results/{}.pickle.dat'

## **1. Model Interpretability**

Boosted decision trees aren't necessarily the most interpretable, but there are a few steps we can take towards a better understanding of what is going on with the models. We can try to gain a better understanding of the model by assessing *global* feature importances as well as inspecting *local* predictions.


### a) Global Feature Importances

XGBoost has a build in feature to plot feature importance. The "F-score" presented here is basically a measure of how often a feature appears in a tree, and is therefore about as basic a feature importance metric as we can get. 


In [ ]:
# Map models to titles
TITLES = ['Baseline',
         'XGB (N=1)',
         'XGB (N=2)',
         'XGB (N=7)',
         'XGB (N=30)',
         'XGB w/ History (N=1)',
         'XGB w/ History (N=2)',
         'XGB w/ History (N=7)',
         'XGB w/ History (N=30)']

MODELS =  ['baseline',
           '50_50_fail_today',
           '50_50_fail_today_or_tomorrow',
           '50_50_fail_this_week',
           '50_50_fail_this_month',
           '50_50_fail_today_history',
           '50_50_fail_today_or_tomorrow_history',
           '50_50_fail_this_week_history',
           '50_50_fail_this_month_history']

TITLE_DICT = dict(zip(MODELS, TITLES))

def get_model(model, model_path):
    """
    Load a saved model so we can use it for predictions
    
    Parameters
    ----------
    model : string
        Name of the model
        
    model_path : string
        Path to the saved model
        
    Returns
    -------
    loaded_model : Model
        Trained Sklearn/xgboost model used to make predictions
    """
    model_file = model_path.format(model)
    loaded_model = pickle.load(open(model_file, "rb"))
    
    return loaded_model


def plot_importances(model, save=False):
    """
    Plot feature importances
    """
    loaded_model = get_model(model, MODEL_PATH)
    plt.figure(figsize=(11,6))
    ax = plt.subplot(1,1,1)
    xgb.plot_importance(loaded_model, 
                        ax=ax, 
                        max_num_features=15, 
                        height=0.8, 
                        show_values=False)
    if 'history' in model:
        labels = [(' ').join(label.get_text().split('_')).title() 
                  for label in ax.get_yticklabels()]
    else:
        labels = [(' ').join(label.get_text().split('_')).title()[:-4] 
                  for label in ax.get_yticklabels()]

    ax.set_yticklabels(labels)
    title = 'Feature Importance for {}'.format(TITLE_DICT[model])
    ax.set_title(title)
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    
    if save:
        plt.tight_layout()
        plt.savefig('feature_importance_{}.png'.format(model))
    
    plt.show()    

In [ ]:
plot_importances('50_50_fail_today_or_tomorrow')

***NOTE:*** Rerun the above cell with the names of different models. 

### b) Shap

The above feature importances do not tell us anything about the diretionality with which each feature contributes to the prediction. However, this can be analysed through Shapley values. Shapley values are a concept borrowed from cooperative game theory. In essense, the shapley values correspond to the contribution of each feature (or "player") towards pushing the prediction (or "outcome") away from the expected value of the model.

Shapley values are problematic, as is the entire concept of trying to "explain" uninterpretable models. Given that interpretability is probably not a critical requirement for this use case, however, we can still use these tools as long as we take the results with a grain of salt. 

In [ ]:
def get_test_data(bucket, prefix):
    """
    Function to get all test data 
    
    Parameters
    ----------
    bucket : str
        Name of bucket in gcs
       
    prefix : str
        Prefix for the test data in bucket
        
    Returns
    -------
    concatenated_df : DataFrame
        A dataframe containing all of the data from the csvs
    """
    # List the relevant files in the bucket
    files = ['gs://' + ('/').join(blob.id.split('/')[:-1]) 
             for blob in storage_client.list_blobs(bucket, 
                                                   prefix=prefix)]
    # Read the files, concatenate them, and sort by date
    df_for_each_blob = (pd.read_csv(file) for file in files)
    concatenated_df = pd.concat(df_for_each_blob, ignore_index=True).sort_values('date')
    # Some drives failed between Jan 1st and Jan 10th, we remove them here
    # (This could also just have been done in BQ ahead of time)
    already_removed_drives = concatenated_df[(concatenated_df['failure'] == 1) 
                                             & (concatenated_df['date'] < '2019-01-10')]['serial_number'].values
    concatenated_df = concatenated_df[~concatenated_df['serial_number'].isin(already_removed_drives)]
    
    return concatenated_df

class PredictionExplainer():
    """
    Class to explain individual predictions of a machine learning model 
    by plotting shap values and displaying features used to make a decision
    """
    def __init__(self, test_data, model):
        self.test_data = test_data
        
        self.model_name = model
        loaded_model = get_model(self.model_name, MODEL_PATH)
        self.model = loaded_model
        
    def _define_window(self, serial_number, date):
        """
        Define a window of data according to the date for which we'd like to explain
        the prediction
        """
        dates = pd.date_range(end=date, periods=10)
        dates = [str(date).split(' ')[0] for date in dates]
        
        return self.test_data[(self.test_data['serial_number'] == serial_number) 
                              & (test_data['date'].isin(dates))]
        
    def _get_features(self, window):
        """
        Get features for a window of data
        """        
        feat_cols = [col for col in self.test_data.columns if 'smart' in col]
        if 'history' in self.model_name:
            aggs = ['last', np.mean, np.var, 'min', 'max']
        else:
            aggs = ['last']
        features = window.groupby('serial_number').agg(['last'])[feat_cols]
        
        return features
    
    def _get_important_features_as_frame(self, window, features, shap_values):
        """
        Get important features according to shap as a dataframe
        """
        idx = np.argsort(abs(shap_values[0]))[-10:]
        top_feats = features.columns[idx[::-1]]
        frame = features[top_feats]
        frame.columns = [('_').join(col) for col in top_feats]
        
        prediction = self.model.predict(features)
        frame['prediction'] = prediction
        frame['failure'] = window['failure'].iloc[-1]

        return frame
        
    def explain_prediction(self, serial_number, date, plot=True, display_frame=True):
        """
        Explain the prediction of an ML model for a particular
        serial number on a particular date
        
        Parameters
        ----------
        serial_number : str
            Name of the hard drive
            
        date : str
            Date of the prediction
            
        plot : bool
            Whether to display a shap summary plot
            
        display_frame : bool
            Whether to display important features as a data frame
            
        Returns
        -------
        """
        window = self._define_window(serial_number, date)
        features = self._get_features(window)
        
        explainer_shap = shap.TreeExplainer(self.model)
        shap_values = explainer_shap.shap_values(features)
        
        self.frame = self._get_important_features_as_frame(window, 
                                                      features, 
                                                      shap_values)
        
        if plot:
            shap.summary_plot(shap_values, features, max_display=10)
        if display_frame:
            display(self.frame)

In [ ]:
test_data = get_test_data(<BUCKET>, prefix='hard-drive-failure/test/test') # your bucket

In [ ]:
# Explainer for the 50_50_fail_today model
xgb_N1_explainer = PredictionExplainer(test_data, '50_50_fail_today')

# Explain individual predictions
xgb_N1_explainer.explain_prediction('S301K6TS', '2019-01-10')
xgb_N1_explainer.explain_prediction('Z305D6G2', '2019-05-17')